线上：0.125866  
需要做特征选择 2018/11/27

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import datetime
import time
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectPercentile,f_regression

d:\Users\onLine\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_1 = pd.read_csv('./data/train_1.csv')
train_2 = pd.read_csv('./data/train_2.csv')
train_3 = pd.read_csv('./data/train_3.csv')
train_4 = pd.read_csv('./data/train_4.csv')

test_1 = pd.read_csv('./data/test_1.csv')
test_2 = pd.read_csv('./data/test_2.csv')
test_3 = pd.read_csv('./data/test_3.csv')
test_4 = pd.read_csv('./data/test_4.csv')

In [3]:
data_1 = pd.concat([train_1,test_1],axis=0,ignore_index=True,sort=False)
data_2 = pd.concat([train_2,test_2],axis=0,ignore_index=True,sort=False)
data_3 = pd.concat([train_3,test_3],axis=0,ignore_index=True,sort=False)
data_4 = pd.concat([train_4,test_4],axis=0,ignore_index=True,sort=False)

In [4]:
pd.options.display.max_columns = None  # 显示的最大列数不受限制

In [5]:
data_1

,时间,辐照度,风速,风向,温度,压强,湿度,实发辐照度,实际功率,id
0,2016-04-01 00:15:00,-1.000000,-0.707547,251,-0.090909,-0.030303,-0.157895,0.00000,-0.019333,NaN
1,2016-04-01 00:15:00,-1.000000,-0.707547,251,-0.090909,-0.030303,-0.157895,0.00000,-0.019333,NaN
2,2016-04-01 00:30:00,-1.000000,-0.716981,250,-0.107071,-0.030303,-0.136842,0.00000,-0.021000,NaN
3,2016-04-01 00:45:00,-1.000000,-0.726415,248,-0.123232,0.030303,-0.094737,0.00000,-0.022000,NaN
4,2016-04-01 01:00:00,-1.000000,-0.735849,244,-0.135354,0.030303,-0.073684,0.00000,-0.022000,NaN
5,2016-04-01 01:15:00,-1.000000,-0.754717,241,-0.147475,0.030303,-0.052632,0.00000,-0.022000,NaN
6,2016-04-01 01:30:00,-1.000000,-0.754717,234,-0.163636,0.030303,-0.010526,0.00000,-0.020667,NaN
7,2016-04-01 01:45:00,-1.000000,-0.754717,231,-0.179798,0.030303,0.010526,0.00000,-0.020667,NaN
8,2016-04-01 02:00:00,-1.000000,-0.764151,227,-0.191919,0.030303,0.031579,0.00000,-0.020667,NaN
9,2016-04-01 02:15:00,-1.000000,-0.764151,220,-0.204040,0.030303,0.073684,0.00000,-0.022000,NaN


In [6]:
data_1

,时间,辐照度,风速,风向,温度,压强,湿度,实发辐照度,实际功率,id
0,2016-04-01 00:15:00,-1.000000,-0.707547,251,-0.090909,-0.030303,-0.157895,0.00000,-0.019333,NaN
1,2016-04-01 00:15:00,-1.000000,-0.707547,251,-0.090909,-0.030303,-0.157895,0.00000,-0.019333,NaN
2,2016-04-01 00:30:00,-1.000000,-0.716981,250,-0.107071,-0.030303,-0.136842,0.00000,-0.021000,NaN
3,2016-04-01 00:45:00,-1.000000,-0.726415,248,-0.123232,0.030303,-0.094737,0.00000,-0.022000,NaN
4,2016-04-01 01:00:00,-1.000000,-0.735849,244,-0.135354,0.030303,-0.073684,0.00000,-0.022000,NaN
5,2016-04-01 01:15:00,-1.000000,-0.754717,241,-0.147475,0.030303,-0.052632,0.00000,-0.022000,NaN
6,2016-04-01 01:30:00,-1.000000,-0.754717,234,-0.163636,0.030303,-0.010526,0.00000,-0.020667,NaN
7,2016-04-01 01:45:00,-1.000000,-0.754717,231,-0.179798,0.030303,0.010526,0.00000,-0.020667,NaN
8,2016-04-01 02:00:00,-1.000000,-0.764151,227,-0.191919,0.030303,0.031579,0.00000,-0.020667,NaN
9,2016-04-01 02:15:00,-1.000000,-0.764151,220,-0.204040,0.030303,0.073684,0.00000,-0.022000,NaN


In [7]:
# f1 = pd.DataFrame(data_4.groupby("风速")["风向"].count()).reset_index().rename(columns={"风向":"count of 风速 and 风向"})
# f2 = pd.DataFrame(data_4.groupby("风速")["风向"].mean()).reset_index().rename(columns={"风向":"mean of 风速 and 风向"})
# f3 = pd.DataFrame(data_4.groupby("风速")["风向"].nunique()).reset_index().rename(columns={"风向":"nuq of 风速 and 风向"})
# f4 = pd.DataFrame(data_4.groupby("湿度")["风向"].mean()).reset_index().rename(columns={"风向":"mean of 湿度 and 风向"})
# f5 = pd.DataFrame(data_4.groupby("风向")["风速"].mean()).reset_index().rename(columns={"风速":"mean of 风向 and 风速"})

In [8]:
# data_4 = pd.merge(data_4,f1, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f2, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f3, how='left', on=["风速"])
# data_4 = pd.merge(data_4,f4, how='left', on=["湿度"])
# data_4 = pd.merge(data_4,f5, how='left', on=["风向"])

In [9]:
data_1

,时间,辐照度,风速,风向,温度,压强,湿度,实发辐照度,实际功率,id
0,2016-04-01 00:15:00,-1.000000,-0.707547,251,-0.090909,-0.030303,-0.157895,0.00000,-0.019333,NaN
1,2016-04-01 00:15:00,-1.000000,-0.707547,251,-0.090909,-0.030303,-0.157895,0.00000,-0.019333,NaN
2,2016-04-01 00:30:00,-1.000000,-0.716981,250,-0.107071,-0.030303,-0.136842,0.00000,-0.021000,NaN
3,2016-04-01 00:45:00,-1.000000,-0.726415,248,-0.123232,0.030303,-0.094737,0.00000,-0.022000,NaN
4,2016-04-01 01:00:00,-1.000000,-0.735849,244,-0.135354,0.030303,-0.073684,0.00000,-0.022000,NaN
5,2016-04-01 01:15:00,-1.000000,-0.754717,241,-0.147475,0.030303,-0.052632,0.00000,-0.022000,NaN
6,2016-04-01 01:30:00,-1.000000,-0.754717,234,-0.163636,0.030303,-0.010526,0.00000,-0.020667,NaN
7,2016-04-01 01:45:00,-1.000000,-0.754717,231,-0.179798,0.030303,0.010526,0.00000,-0.020667,NaN
8,2016-04-01 02:00:00,-1.000000,-0.764151,227,-0.191919,0.030303,0.031579,0.00000,-0.020667,NaN
9,2016-04-01 02:15:00,-1.000000,-0.764151,220,-0.204040,0.030303,0.073684,0.00000,-0.022000,NaN


In [10]:
# 返回时间戳
def get_timestamp(x):
    try:
        t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f")
        t = int(round((t.timestamp())))  # 四舍五入，去掉毫秒级  
    except:
        t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
        t = int(t.timestamp())
    return t
    

In [11]:
data_1['时间戳'] = data_1['时间'].map(lambda x: get_timestamp(x))

In [12]:
# start_timestamp = data_1['时间戳'][0]
# loss_time = []
# timestampe = data_1['时间戳']
# for i in range(1,len(timestampe)):
#     if timestampe[i] == timestampe[i-1]:
#         continue
#     start_timestamp += 900
#     while timestampe[i] != start_timestamp:
#         loss_time.append(start_timestamp)
#         start_timestamp += 900    

# loss_time
# # 缺失了6101条数据
# len(loss_time)

In [13]:
data = data_1[['时间戳','辐照度','风速','风向','温度','压强','湿度']]
data = data.drop_duplicates(['时间戳'])  # 去掉重复值，不去掉合并会多行

t1 = data.copy()
t1.columns = ['时间戳','辐照度-1','风速-1','风向-1','温度-1','压强-1','湿度-1']
t1_0 = t1[:1].copy() # 获取第一行，用于合并
t1['时间戳'] = t1['时间戳'].map(lambda x: x+900)  #15分*60=900秒
t1 = pd.concat([t1_0,t1],axis=0)
data_1 = pd.merge(data_1,t1,how='left',on='时间戳')

# t2 = data.copy()
# t2.columns = ['时间戳','辐照度-2','风速-2','风向-2','温度-2','压强-2','湿度-2']
# t2_0 = t2[:2].copy() # 获取第一行，用于合并
# t2['时间戳'] = t2['时间戳'].map(lambda x: x+1800)  #30分*60=1800秒
# t2 = pd.concat([t2_0,t2],axis=0)
# data_1 = pd.merge(data_1,t2,how='left',on='时间戳')


t1 = data.copy()
t1.columns = ['时间戳','辐照度+1','风速+1','风向+1','温度+1','压强+1','湿度+1']
t1_0 = t1[-1:].copy() # 获取最后一行，用于合并
t1['时间戳'] = t1['时间戳'].map(lambda x: x-900)  #15分*60=900秒
t1 = pd.concat([t1,t1_0],axis=0)
data_1 = pd.merge(data_1,t1,how='left',on='时间戳')

t2 = data.copy()
t2.columns = ['时间戳','辐照度+2','风速+2','风向+2','温度+2','压强+2','湿度+2']
t2_0 = t2[-2:].copy() # 获取最后两行，用于合并
t2['时间戳'] = t2['时间戳'].map(lambda x: x-1800)  #30分*60=900秒
t2 = pd.concat([t2,t2_0],axis=0)
data_1 = pd.merge(data_1,t2,how='left',on='时间戳')

t3 = data.copy()
t3.columns = ['时间戳','辐照度+3','风速+3','风向+3','温度+3','压强+3','湿度+3']
t3_0 = t3[-3:].copy() # 获取最后三行，用于合并
t3['时间戳'] = t3['时间戳'].map(lambda x: x-2700)  #45分*60=2700秒
t3 = pd.concat([t3,t3_0],axis=0)
data_1 = pd.merge(data_1,t3,how='left',on='时间戳')

# t4 = data.copy()
# t4.columns = ['时间戳','辐照度+4','风速+4','风向+4','温度+4','压强+4','湿度+4']
# t4_0 = t4[-4:].copy() # 获取第一行，用于合并
# t4['时间戳'] = t4['时间戳'].map(lambda x: x-3600)  #60分*60=2700秒
# t4 = pd.concat([t4,t4_0],axis=0)
# data_1 = pd.merge(data_1,t4,how='left',on='时间戳')

In [14]:
data_2['时间戳'] = data_2['时间'].map(lambda x: get_timestamp(x))
data = data_2[['时间戳','辐照度','风速','风向','温度','压强','湿度']]
data = data.drop_duplicates(['时间戳'])  # 去掉重复值，不去掉合并会多行

t1 = data.copy()
t1.columns = ['时间戳','辐照度-1','风速-1','风向-1','温度-1','压强-1','湿度-1']
t1_0 = t1[:1].copy() # 获取第一行，用于合并
t1['时间戳'] = t1['时间戳'].map(lambda x: x+900)  #15分*60=900秒
t1 = pd.concat([t1_0,t1],axis=0)
data_2 = pd.merge(data_2,t1,how='left',on='时间戳')

# t2 = data.copy()
# t2.columns = ['时间戳','辐照度-2','风速-2','风向-2','温度-2','压强-2','湿度-2']
# t2_0 = t2[:2].copy() # 获取第一行，用于合并
# t2['时间戳'] = t2['时间戳'].map(lambda x: x+1800)  #30分*60=1800秒
# t2 = pd.concat([t2_0,t2],axis=0)
# data_2 = pd.merge(data_2,t2,how='left',on='时间戳')

t1 = data.copy()
t1.columns = ['时间戳','辐照度+1','风速+1','风向+1','温度+1','压强+1','湿度+1']
t1_0 = t1[-1:].copy() # 获取最后一行，用于合并
t1['时间戳'] = t1['时间戳'].map(lambda x: x-900)  #15分*60=900秒
t1 = pd.concat([t1,t1_0],axis=0)
data_2 = pd.merge(data_2,t1,how='left',on='时间戳')

# t2 = data.copy()
# t2.columns = ['时间戳','辐照度+2','风速+2','风向+2','温度+2','压强+2','湿度+2']
# t2_0 = t2[-2:].copy() # 获取第一行，用于合并
# t2['时间戳'] = t2['时间戳'].map(lambda x: x-1800)  #15分*60=900秒
# t2 = pd.concat([t2,t2_0],axis=0)
# data_2 = pd.merge(data_2,t2,how='left',on='时间戳')

In [15]:
data_3['时间戳'] = data_3['时间'].map(lambda x: get_timestamp(x))
# data = data_3[['时间戳','辐照度','风速','风向','温度','压强','湿度']]
# data = data.drop_duplicates(['时间戳'])  # 去掉重复值，不去掉合并会多行

# t1 = data.copy()
# t1.columns = ['时间戳','辐照度-1','风速-1','风向-1','温度-1','压强-1','湿度-1']
# t1_0 = t1[:1].copy() # 获取第一行，用于合并
# t1['时间戳'] = t1['时间戳'].map(lambda x: x+900)  #15分*60=900秒
# t1 = pd.concat([t1_0,t1],axis=0)
# data_3 = pd.merge(data_3,t1,how='left',on='时间戳')

# t2 = data.copy()
# t2.columns = ['时间戳','辐照度-2','风速-2','风向-2','温度-2','压强-2','湿度-2']
# t2_0 = t2[:2].copy() # 获取第一行，用于合并
# t2['时间戳'] = t2['时间戳'].map(lambda x: x+1800)  #30分*60=1800秒
# t2 = pd.concat([t2_0,t2],axis=0)
# data_2 = pd.merge(data_2,t2,how='left',on='时间戳')

# # 
# t1 = data.copy()
# t1.columns = ['时间戳','辐照度+1','风速+1','风向+1','温度+1','压强+1','湿度+1']
# t1_0 = t1[-1:].copy() # 获取第一行，用于合并
# t1['时间戳'] = t1['时间戳'].map(lambda x: x-900)  #15分*60=900秒
# t1 = pd.concat([t1,t1_0],axis=0)
# data_3 = pd.merge(data_3,t1,how='left',on='时间戳')

In [16]:
data_4['时间戳'] = data_4['时间'].map(lambda x: get_timestamp(x))
# data = data_4[['时间戳','辐照度','风速','风向','温度','压强','湿度']]
# data = data.drop_duplicates(['时间戳'])  # 去掉重复值，不去掉合并会多行

# t1 = data.copy()
# t1.columns = ['时间戳','辐照度-1','风速-1','风向-1','温度-1','压强-1','湿度-1']
# t1_0 = t1[:1].copy() # 获取第一行，用于合并
# t1['时间戳'] = t1['时间戳'].map(lambda x: x+900)  #15分*60=900秒
# t1 = pd.concat([t1_0,t1],axis=0)
# data_4 = pd.merge(data_4,t1,how='left',on='时间戳')

# t1 = data.copy()
# t1.columns = ['时间戳','辐照度+1','风速+1','风向+1','温度+1','压强+1','湿度+1']
# t1_0 = t1[-1:].copy() # 获取第一行，用于合并
# t1['时间戳'] = t1['时间戳'].map(lambda x: x-900)  #15分*60=900秒
# t1 = pd.concat([t1,t1_0],axis=0)
# data_4 = pd.merge(data_4,t1,how='left',on='时间戳')

In [17]:
# index_list = data_1[data_1['辐照度-1'].isnull()].index.tolist()
# for index in index_list:
#     for feat in ['辐照度','风速','风向','温度','压强','湿度']:
#         data_1.loc[index,feat+'-1'] = data_1.loc[index,feat]

# index_list = data_1[data_1['辐照度+1'].isnull()].index.tolist()
# for index in index_list:
#     for feat in ['辐照度','风速','风向','温度','压强','湿度']:
#         data_1.loc[index,feat+'+1'] = data_1.loc[index,feat]
        
# 用最近的时刻去填补
index_list = data_1[data_1['辐照度+2'].isnull()].index.tolist()
for index in index_list:
    for feat in ['辐照度','风速','风向','温度','压强','湿度']:
        data_1.loc[index,feat+'+2'] = data_1.loc[index,feat+'+1']
        
# # 用最近的时刻去填补
# index_list = data_1[data_1['辐照度+3'].isnull()].index.tolist()
# for index in index_list:
#     for feat in ['辐照度','风速','风向','温度','压强','湿度']:
#         data_1.loc[index,feat+'+3'] = data_1.loc[index,feat+'+2']

In [18]:
# data_1['辐照度-辐照度+1'] = data_1['辐照度'] - data_1['辐照度+1']
# data_1['辐照度-辐照度+3'] = data_1['辐照度'] - data_1['辐照度+3']

In [19]:
# # 填充缺失值
# basic_features = ['辐照度','风速','风向','温度','压强','湿度']
# for i in len(data_1):
    
# for feat in basic_features:
#     data_1.loc[data_1[feat].isnull(),feat+'-1'] = data_1.loc[0,feat]
    
# data_1

In [20]:
# 将带有毫秒级时间四舍五入为秒级时间
# 输入字符串时间，"%Y-%m-%d %H:%M:%S.%f"
# 输出datetime格式的时间，年月日时分秒

def get_new_time(x):
    try:
        t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f")
        t = round(t.timestamp())  # 四舍五入，去掉毫秒级
        t = datetime.datetime.fromtimestamp(t)  # 转换为datetime格式
    except:
        t = datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")  # 转换为datetime格式
    return t

In [21]:
data_1['year'] = data_1['时间'].map(lambda x: get_new_time(x).year)
data_1['month'] = data_1['时间'].map(lambda x: get_new_time(x).month)
data_1['day'] = data_1['时间'].map(lambda x: get_new_time(x).day)
data_1['hour'] = data_1['时间'].map(lambda x: get_new_time(x).hour)
data_1['minute'] = data_1['时间'].map(lambda x: get_new_time(x).minute)

data_2['year'] = data_2['时间'].map(lambda x: get_new_time(x).year)
data_2['month'] = data_2['时间'].map(lambda x: get_new_time(x).month)
data_2['day'] = data_2['时间'].map(lambda x: get_new_time(x).day)
data_2['hour'] = data_2['时间'].map(lambda x: get_new_time(x).hour)
data_2['minute'] = data_2['时间'].map(lambda x: get_new_time(x).minute)

data_3['year'] = data_3['时间'].map(lambda x: get_new_time(x).year)
data_3['month'] = data_3['时间'].map(lambda x: get_new_time(x).month)
data_3['day'] = data_3['时间'].map(lambda x: get_new_time(x).day)
data_3['hour'] = data_3['时间'].map(lambda x: get_new_time(x).hour)
data_3['minute'] = data_3['时间'].map(lambda x: get_new_time(x).minute)

data_4['year'] = data_4['时间'].map(lambda x: get_new_time(x).year)
data_4['month'] = data_4['时间'].map(lambda x: get_new_time(x).month)
data_4['day'] = data_4['时间'].map(lambda x: get_new_time(x).day)
data_4['hour'] = data_4['时间'].map(lambda x: get_new_time(x).hour)
data_4['minute'] = data_4['时间'].map(lambda x: get_new_time(x).minute)

In [22]:
data_1

,时间,辐照度,风速,风向,温度,压强,湿度,实发辐照度,实际功率,id,时间戳,辐照度-1,风速-1,风向-1,温度-1,压强-1,湿度-1,辐照度+1,风速+1,风向+1,温度+1,压强+1,湿度+1,辐照度+2,风速+2,风向+2,温度+2,压强+2,湿度+2,辐照度+3,风速+3,风向+3,温度+3,压强+3,湿度+3,year,month,day,hour,minute
0,2016-04-01 00:15:00,-1.000000,-0.707547,251,-0.090909,-0.030303,-0.157895,0.00000,-0.019333,NaN,1459440900,-1.000000,-0.707547,251.0,-0.090909,-0.030303,-0.157895,-1.000000,-0.716981,250.0,-0.107071,-0.030303,-0.136842,-1.000000,-0.726415,248.0,-0.123232,0.030303,-0.094737,-1.000000,-0.735849,244.0,-0.135354,0.030303,-0.073684,2016,4,1,0,15
1,2016-04-01 00:15:00,-1.000000,-0.707547,251,-0.090909,-0.030303,-0.157895,0.00000,-0.019333,NaN,1459440900,-1.000000,-0.707547,251.0,-0.090909,-0.030303,-0.157895,-1.000000,-0.716981,250.0,-0.107071,-0.030303,-0.136842,-1.000000,-0.726415,248.0,-0.123232,0.030303,-0.094737,-1.000000,-0.735849,244.0,-0.135354,0.030303,-0.073684,2016,4,1,0,15
2,2016-04-01 00:30:00,-1.000000,-0.716981,250,-0.107071,-0.030303,-0.136842,0.00000,-0.021000,NaN,1459441800,-1.000000,-0.707547,251.0,-0.090909,-0.030303,-0.157895,-1.000000,-0.726415,248.0,-0.123232,0.030303,-0.094737,-1.000000,-0.735849,244.0,-0.135354,0.030303,-0.073684,-1.000000,-0.754717,241.0,-0.147475,0.030303,-0.052632,2016,4,1,0,30
3,2016-04-01 00:45:00,-1.000000,-0.726415,248,-0.123232,0.030303,-0.094737,0.00000,-0.022000,NaN,1459442700,-1.000000,-0.716981,250.0,-0.107071,-0.030303,-0.136842,-1.000000,-0.735849,244.0,-0.135354,0.030303,-0.073684,-1.000000,-0.754717,241.0,-0.147475,0.030303,-0.052632,-1.000000,-0.754717,234.0,-0.163636,0.030303,-0.010526,2016,4,1,0,45
4,2016-04-01 01:00:00,-1.000000,-0.735849,244,-0.135354,0.030303,-0.073684,0.00000,-0.022000,NaN,1459443600,-1.000000,-0.726415,248.0,-0.123232,0.030303,-0.094737,-1.000000,-0.754717,241.0,-0.147475,0.030303,-0.052632,-1.000000,-0.754717,234.0,-0.163636,0.030303,-0.010526,-1.000000,-0.754717,231.0,-0.179798,0.030303,0.010526,2016,4,1,1,0
5,2016-04-01 01:15:00,-1.000000,-0.754717,241,-0.147475,0.030303,-0.052632,0.00000,-0.022000,NaN,1459444500,-1.000000,-0.735849,244.0,-0.135354,0.030303,-0.073684,-1.000000,-0.754717,234.0,-0.163636,0.030303,-0.010526,-1.000000,-0.754717,231.0,-0.179798,0.030303,0.010526,-1.000000,-0.764151,227.0,-0.191919,0.030303,0.031579,2016,4,1,1,15
6,2016-04-01 01:30:00,-1.000000,-0.754717,234,-0.163636,0.030303,-0.010526,0.00000,-0.020667,NaN,1459445400,-1.000000,-0.754717,241.0,-0.147475,0.030303,-0.052632,-1.000000,-0.754717,231.0,-0.179798,0.030303,0.010526,-1.000000,-0.764151,227.0,-0.191919,0.030303,0.031579,-1.000000,-0.764151,220.0,-0.204040,0.030303,0.073684,2016,4,1,1,30
7,2016-04-01 01:45:00,-1.000000,-0.754717,231,-0.179798,0.030303,0.010526,0.00000,-0.020667,NaN,1459446300,-1.000000,-0.754717,234.0,-0.163636,0.030303,-0.010526,-1.000000,-0.764151,227.0,-0.191919,0.030303,0.031579,-1.000000,-0.764151,220.0,-0.204040,0.030303,0.073684,-1.000000,-0.773585,215.0,-0.216162,0.030303,0.094737,2016,4,1,1,45
8,2016-04-01 02:00:00,-1.000000,-0.764151,227,-0.191919,0.030303,0.031579,0.00000,-0.020667,NaN,1459447200,-1.000000,-0.754717,231.0,-0.179798,0.030303,0.010526,-1.000000,-0.764151,220.0,-0.204040,0.030303,0.073684,-1.000000,-0.773585,215.0,-0.216162,0.030303,0.094737,-1.000000,-0.773585,210.0,-0.224242,0.030303,0.115789,2016,4,1,2,0
9,2016-04-01 02:15:00,-1.000000,-0.764151,220,-0.204040,0.030303,0.073684,0.00000,-0.022000,NaN,1459448100,-1.000000,-0.764151,227.0,-0.191919,0.030303,0.031579,-1.000000,-0.773585,215.0,-0.216162,0.030303,0.094737,-1.000000,-0.773585,210.0,-0.224242,0.030303,0.115789,-1.000000,-0.773585,204.0,-0.236364,0.030303,0.136842,2016,4,1,2,15


In [23]:
data_1['风向_count']= data_1['风向'].map(data_1['风向'].value_counts()).astype(int)
# data_1['风向'].value_counts()
data_1['风向_count'] = (data_1['风向_count']-data_1['风向_count'].min()) / (data_1['风向_count'].max()-data_1['风向_count'].min())

In [24]:
# data_1['风向-1_count']= data_1['风向-1'].map(data_1['风向-1'].value_counts())
# # data_1['风向'].value_counts()
# data_1['风向-1_count'] = (data_1['风向-1_count']-data_1['风向-1_count'].min()) / (data_1['风向-1_count'].max()-data_1['风向-1_count'].min())

# data_4['风向_count']= data_4['风向'].map(data_4['风向'].value_counts()).astype(int)
# # data_1['风向'].value_counts()
# data_4['风向_count'] = (data_4['风向_count']-data_4['风向_count'].min()) / (data_4['风向_count'].max()-data_4['风向_count'].min())

In [25]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时最小':'min'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day'])[item].agg({item+'日最小':'min'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month'])[item].agg({item+'月最小':'min'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])


In [26]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month','day','hour'])[item].agg({item+'时最小':'min'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month','day','hour'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month','day'])[item].agg({item+'日最小':'min'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month','day'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month'])[item].agg({item+'月最小':'min'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month'])

In [27]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_3.groupby(['year','month','day','hour'])[item].agg({item+'时最小':'min'})
#     data_3 = pd.merge(data_3,t1,how='left',on=['year','month','day','hour'])
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_3.groupby(['year','month','day'])[item].agg({item+'日最小':'min'})
    data_3 = pd.merge(data_3,t1,how='left',on=['year','month','day'])
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_3.groupby(['year','month'])[item].agg({item+'月最小':'min'})
    data_3 = pd.merge(data_3,t1,how='left',on=['year','month'])

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """
d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [28]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_4.groupby(['year','month','day','hour'])[item].agg({item+'时最小':'min'})
#     data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day','hour'])
    
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month','day'])[item].agg({item+'日最小':'min'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day'])
    
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month'])[item].agg({item+'月最小':'min'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month'])

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


In [29]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时中值':'median'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
    
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day'])[item].agg({item+'日中值':'median'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month'])[item].agg({item+'月中值':'median'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [30]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month','day','hour'])[item].agg({item+'时中值':'median'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month','day','hour'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month','day'])[item].agg({item+'日中值':'median'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month','day'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month'])[item].agg({item+'月中值':'median'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month'])

In [31]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_3.groupby(['year','month','day','hour'])[item].agg({item+'时中值':'median'})
    data_3 = pd.merge(data_3,t1,how='left',on=['year','month','day','hour'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_3.groupby(['year','month','day'])[item].agg({item+'日中值':'median'})
#     data_3 = pd.merge(data_3,t1,how='left',on=['year','month','day'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_3.groupby(['year','month'])[item].agg({item+'月中值':'median'})
#     data_3 = pd.merge(data_3,t1,how='left',on=['year','month'])

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [32]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month','day','hour'])[item].agg({item+'时中值':'median'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day','hour'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_4.groupby(['year','month','day'])[item].agg({item+'日中值':'median'})
#     data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day'])
    
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_4.groupby(['year','month'])[item].agg({item+'月中值':'median'})
#     data_4 = pd.merge(data_4,t1,how='left',on=['year','month'])

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [33]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day','hour'])[item].agg({item+'时最大':'max'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day'])[item].agg({item+'日最大':'max'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month'])[item].agg({item+'月最大':'max'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])
#     data_1[item+'时最大'] = data_1[item+'时最大'] - data_1[item]

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [34]:
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month','day','hour'])[item].agg({item+'时最大':'max'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month','day','hour'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month','day'])[item].agg({item+'日最大':'max'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month','day'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month'])[item].agg({item+'月最大':'max'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month'])

In [35]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_3.groupby(['year','month','day','hour'])[item].agg({item+'时最大':'max'})
    data_3 = pd.merge(data_3,t1,how='left',on=['year','month','day','hour'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_3.groupby(['year','month','day'])[item].agg({item+'日最大':'max'})
#     data_3 = pd.merge(data_3,t1,how='left',on=['year','month','day'])
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_3.groupby(['year','month'])[item].agg({item+'月最大':'max'})
#     data_3 = pd.merge(data_3,t1,how='left',on=['year','month'])

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [36]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month','day','hour'])[item].agg({item+'时最大':'max'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day','hour'])
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month','day'])[item].agg({item+'日最大':'max'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day'])


for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month'])[item].agg({item+'月最大':'max'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month'])

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """
d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.


In [37]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month','day','hour'])[item].agg('mean').reset_index().rename(columns={item:item+'时平均'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day','hour'])
    data_1[item+'时平均'] = data_1[item] - data_1[item+'时平均']
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_1.groupby(['year','month','day'])[item].agg('mean').reset_index().rename(columns={item:item+'日平均'})
#     data_1 = pd.merge(data_1,t1,how='left',on=['year','month','day'])
#     data_1[item+'日平均'] = data_1[item] - data_1[item+'日平均']
# data_1.drop(['year'],axis=1,inplace=True)
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_1.groupby(['year','month'])[item].agg('mean').reset_index().rename(columns={item:item+'月平均'})
    data_1 = pd.merge(data_1,t1,how='left',on=['year','month'])
    data_1[item+'月平均'] = data_1[item] - data_1[item+'月平均']
data_1.drop(['year'],axis=1,inplace=True)

In [38]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_2.groupby(['year','month','day','hour'])[item].agg('mean').reset_index().rename(columns={item:item+'时平均'})
    data_2 = pd.merge(data_2,t1,how='left',on=['year','month','day','hour'])
    data_2[item+'时平均'] = data_2[item] - data_2[item+'时平均']

# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_2.groupby(['year','month','day'])[item].agg('mean').reset_index().rename(columns={item:item+'日平均'})
#     data_2 = pd.merge(data_2,t1,how='left',on=['year','month','day'])
#     data_2[item+'日平均'] = data_2[item] - data_2[item+'日平均']
    
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_2.groupby(['year','month'])[item].agg('mean').reset_index().rename(columns={item:item+'月平均'})
    data_2 = pd.merge(data_2,t1,how='left',on=['year','month'])
    data_2[item+'月平均'] = data_2[item] - data_2[item+'月平均']
data_2.drop(['year','month','day','hour','minute'],axis=1,inplace=True)

In [39]:
# data_1
# # 新加没试，2018/11/23 22：54
# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_3.groupby(['year','month','day','hour'])[item].agg('mean').reset_index().rename(columns={item:item+'时平均'})
#     data_3 = pd.merge(data_3,t1,how='left',on=['year','month','day','hour'])
#     data_3[item+'时平均'] = data_3[item] - data_3[item+'时平均']
    
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_3.groupby(['year','month','day'])[item].agg('mean').reset_index().rename(columns={item:item+'日平均'})
    data_3 = pd.merge(data_3,t1,how='left',on=['year','month','day'])
    data_3[item+'日平均'] = data_3[item] - data_3[item+'日平均']
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_3.groupby(['year','month'])[item].agg('mean').reset_index().rename(columns={item:item+'月平均'})
    data_3 = pd.merge(data_3,t1,how='left',on=['year','month'])
    data_3[item+'月平均'] = data_3[item] - data_3[item+'月平均']
data_3.drop(['year','month','day','hour','minute'],axis=1,inplace=True)

# for item in ['辐照度','风速','温度','压强','湿度']:
#     t1 = data_4.groupby(['year','month','day'])[item].agg('mean').reset_index().rename(columns={item:item+'日平均'})
#     data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day'])
#     data_4[item+'日平均'] = data_4[item] - data_4[item+'日平均']
# # data_4.drop(['year','month','day','hour','minute'],axis=1,inplace=True)

In [40]:
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month','day','hour'])[item].agg('mean').reset_index().rename(columns={item:item+'时平均'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day','hour'])
    data_4[item+'时平均'] = data_4[item] - data_4[item+'时平均']

for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month','day'])[item].agg('mean').reset_index().rename(columns={item:item+'日平均'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month','day'])
    data_4[item+'日平均'] = data_4[item] - data_4[item+'日平均']
    
for item in ['辐照度','风速','温度','压强','湿度']:
    t1 = data_4.groupby(['year','month'])[item].agg('mean').reset_index().rename(columns={item:item+'月平均'})
    data_4 = pd.merge(data_4,t1,how='left',on=['year','month'])
    data_4[item+'月平均'] = data_4[item] - data_4[item+'月平均']
data_4.drop(['year','month','day','hour','minute'],axis=1,inplace=True)

In [41]:
# data_3

In [42]:
# for item in [ '辐照度','风速','温度','压强','湿度']:
# #     data_1[item+'-平均'] = data_1[item] - data_1[item].mean()
#     data_2[item+'-平均'] = data_2[item] - data_2[item].mean()
# #     data_3[item+'-平均'] = data_3[item] - data_3[item].mean()
#     data_4[item+'-平均'] = data_4[item] - data_4[item].mean()

# feature = ['辐照度+1','风速+1','温度+1','压强+1','湿度+1']
# for item in feature:
#     data_2[item+'-平均'] = data_2[item] - data_2[item].mean()
    
# for item in [ '辐照度','风速','温度','压强','湿度']:
# #     data_1[item+'-min'] = data_1[item] - data_1[item].min()
#     data_2[item+'-min'] = data_2[item] - data_2[item].min()
# #     data_3[item+'-min'] = data_3[item] - data_3[item].min()
#     data_4[item+'-min'] = data_4[item] - data_4[item].min()

In [43]:
# column_names = ['辐照度','温度','压强','湿度']
# poly_transformer = PolynomialFeatures(degree=2,interaction_only=False,include_bias=False)
# poly_data = pd.DataFrame(poly_transformer.fit_transform(data_4[column_names]),columns=poly_transformer.get_feature_names(column_names))
# poly_data.drop(column_names,axis=1,inplace=True)
# data_4 = pd.concat([data_4,poly_data],axis=1,sort=False)

# poly_transformer = PolynomialFeatures(degree=2,interaction_only=False,include_bias=False)
# poly_data = pd.DataFrame(poly_transformer.fit_transform(data_1[column_names]),columns=poly_transformer.get_feature_names(column_names))
# poly_data.drop(column_names,axis=1,inplace=True)
# data_1 = pd.concat([data_1,poly_data],axis=1,sort=False)

In [44]:
# data_1 = pd.concat([data_1,dummies],axis=1,sort=False)
# data_1

In [45]:
# dummies = pd.get_dummies(data_2['风向'],prefix='风向')
# data_2 = pd.concat([data_2,dummies],axis=1,sort=False)

# dummies = pd.get_dummies(data_3['风向'],prefix='风向')
# data_3 = pd.concat([data_3,dummies],axis=1,sort=False)

# dummies = pd.get_dummies(data_4['风向'],prefix='风向')
# data_4 = pd.concat([data_4,dummies],axis=1,sort=False)

In [46]:
# 距离中心13点30分(13.5)的时间距离
data_1['dis2peak'] = data_1['时间'].map(lambda x: (13.5-abs(13.5-get_new_time(x).hour))/13.5)
# data_2['dis2peak'] = data_2['时间'].map(lambda x: (13.5-abs(13.5-get_new_time(x).hour))/13.5)
# data_3['dis2peak'] = data_3['hour'].map(lambda x: (13.5-abs(13.5-x))/13.5)
data_4['dis2peak'] = data_4['时间'].map(lambda x: (13.5-abs(13.5-get_new_time(x).hour))/13.5)

In [47]:
def map_hour(h):
    if h < 7:
        return 1
    elif h < 10:
        return 2
    elif h < 13:
        return 3
    elif h < 16:
        return 4
    elif h < 19:
        return 5
    else:
        return 1

In [48]:
data_1['maphour'] = data_1['时间'].map(lambda x: map_hour(get_new_time(x).hour))
# data_2['maphour'] = data_2['时间'].map(lambda x:map_hour(get_new_time(x).hour))
# data_3['maphour'] = data_3['hour'].map(map_hour)
# data_4['maphour'] = data_4['时间'].map(lambda x:map_hour(get_new_time(x).hour))

In [49]:
X_train_1 = data_1[data_1['实际功率'].notnull()].drop(['时间','实发辐照度','实际功率','id'],axis=1)
y_train_1 = data_1[data_1['实际功率'].notnull()]['实际功率']
X_test_1 = data_1[data_1['实际功率'].isnull()].drop(['时间','实发辐照度','实际功率','id'],axis=1)

In [50]:
# X_train_1 = data_1[:len(train_1)].drop(['时间','实发辐照度','实际功率','id','时间戳'],axis=1)

# y_train_1 = data_1[:len(train_1)]['实际功率']

# X_test_1 = data_1[len(train_1):].drop(['时间','实发辐照度','实际功率','id','时间戳'],axis=1)

In [51]:
train_1_features = X_train_1.columns
train_1_features

Index(['辐照度', '风速', '风向', '温度', '压强', '湿度', '时间戳', '辐照度-1', '风速-1', '风向-1',
       '温度-1', '压强-1', '湿度-1', '辐照度+1', '风速+1', '风向+1', '温度+1', '压强+1', '湿度+1',
       '辐照度+2', '风速+2', '风向+2', '温度+2', '压强+2', '湿度+2', '辐照度+3', '风速+3',
       '风向+3', '温度+3', '压强+3', '湿度+3', 'month', 'day', 'hour', 'minute',
       '风向_count', '辐照度日中值', '风速日中值', '温度日中值', '压强日中值', '湿度日中值', '辐照度时最大',
       '风速时最大', '温度时最大', '压强时最大', '湿度时最大', '辐照度时平均', '风速时平均', '温度时平均', '压强时平均',
       '湿度时平均', '辐照度月平均', '风速月平均', '温度月平均', '压强月平均', '湿度月平均', 'dis2peak',
       'maphour'],
      dtype='object')

In [52]:
prediction_1 = test_1[['id']]

X_train_1_loc = X_train_1.values
X_test_1_loc = X_test_1.values
y_train_1_loc = y_train_1.values

# SKB = SelectPercentile(f_regression,percentile=95).fit(X_train_1_loc,y_train_1_loc)
# X_train_1_loc = SKB.transform(X_train_1_loc)
# X_test_1_loc = SKB.transform(X_test_1_loc)

In [53]:
len(X_train_1)

66859

In [54]:
len(X_test_1)

11808

In [55]:
len(y_train_1_loc)

66859

In [56]:
len(y_train_1_loc[y_train_1_loc>0])

33412

In [57]:
from sklearn.metrics import mean_absolute_error
def my_metric_1(y_true,y_pred): 
    y_true_1 = y_true[y_true>=10*0.03]
    y_pred_1 = y_pred[y_true>=10*0.03]
    score = mean_absolute_error(y_true_1,y_pred_1)/10.0
    return 'my_metric_1',score,False  # False表示score不是越大越好(is_bigger_better)

In [58]:
model_1 = lgb.LGBMRegressor('gbdt',num_leaves=51,max_depth=-1,learning_rate=0.05,n_estimators=100,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.001,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)

# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_train_1_loc, n_folds=10, shuffle=True, random_state=2018))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model_1.fit(X_train_1_loc[train_index], y_train_1_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_1_loc[train_index], y_train_1_loc[train_index]), 
                                    (X_train_1_loc[test_index], y_train_1_loc[test_index])],
                          eval_metric=my_metric_1,
                          early_stopping_rounds=50)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_1'])
    loss += lgb_model.best_score_['valid']['my_metric_1']
    test_pred= lgb_model.predict(X_test_1_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_1['predict_%s' % str(i)] = test_pred
print('my_metric_1:', baseloss, loss/10)

p = prediction_1.drop(['id'],axis=1)
prediction_1['prediction']=p.mean(axis=1)


now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')

import csv
import codecs

baseloss.append(np.mean(baseloss))
baseloss.insert(0,now)
file_csv = codecs.open('./result/model_1_loss.csv','a+','utf-8',buffering=0)#追加，缓存为0
# writer = csv.writer(file_csv, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
writer = csv.writer(file_csv)
writer.writerow(baseloss)

d:\Users\onLine\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Fold 0
[1]	train's l2: 8.17362	train's my_metric_1: 0.294816	valid's l2: 8.01465	valid's my_metric_1: 0.292057
Training until validation scores don't improve for 50 rounds.
[2]	train's l2: 7.47078	train's my_metric_1: 0.282223	valid's l2: 7.32755	valid's my_metric_1: 0.279684
[3]	train's l2: 6.83522	train's my_metric_1: 0.27041	valid's l2: 6.70976	valid's my_metric_1: 0.26819
[4]	train's l2: 6.26068	train's my_metric_1: 0.25934	valid's l2: 6.15091	valid's my_metric_1: 0.257357
[5]	train's l2: 5.74134	train's my_metric_1: 0.248952	valid's l2: 5.64642	valid's my_metric_1: 0.247213
[6]	train's l2: 5.27271	train's my_metric_1: 0.23923	valid's l2: 5.18909	valid's my_metric_1: 0.237662
[7]	train's l2: 4.84838	train's my_metric_1: 0.230102	valid's l2: 4.77782	valid's my_metric_1: 0.228743
[8]	train's l2: 4.46509	train's my_metric_1: 0.221514	valid's l2: 4.40406	valid's my_metric_1: 0.220204
[9]	train's l2: 4.11827	train's my_metric_1: 0.213473	valid's l2: 4.06664	valid's my_metric_1: 0.212273

[78]	train's l2: 0.581479	train's my_metric_1: 0.0850019	valid's l2: 0.673269	valid's my_metric_1: 0.0903925
[79]	train's l2: 0.578232	train's my_metric_1: 0.084698	valid's l2: 0.669848	valid's my_metric_1: 0.0901097
[80]	train's l2: 0.574752	train's my_metric_1: 0.0844182	valid's l2: 0.66704	valid's my_metric_1: 0.0898716
[81]	train's l2: 0.572314	train's my_metric_1: 0.0842059	valid's l2: 0.664814	valid's my_metric_1: 0.0896819
[82]	train's l2: 0.568805	train's my_metric_1: 0.0838706	valid's l2: 0.661734	valid's my_metric_1: 0.0893871
[83]	train's l2: 0.565527	train's my_metric_1: 0.0835923	valid's l2: 0.658856	valid's my_metric_1: 0.0891705
[84]	train's l2: 0.561729	train's my_metric_1: 0.0832699	valid's l2: 0.655578	valid's my_metric_1: 0.0889014
[85]	train's l2: 0.558838	train's my_metric_1: 0.0830192	valid's l2: 0.653184	valid's my_metric_1: 0.0887276
[86]	train's l2: 0.554789	train's my_metric_1: 0.0826758	valid's l2: 0.649171	valid's my_metric_1: 0.0883777
[87]	train's l2: 0.55

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[1]	train's l2: 8.15922	train's my_metric_1: 0.294354	valid's l2: 8.14382	valid's my_metric_1: 0.298187
Training until validation scores don't improve for 50 rounds.
[2]	train's l2: 7.4582	train's my_metric_1: 0.281791	valid's l2: 7.44008	valid's my_metric_1: 0.285347
[3]	train's l2: 6.82443	train's my_metric_1: 0.269979	valid's l2: 6.80378	valid's my_metric_1: 0.273272
[4]	train's l2: 6.25145	train's my_metric_1: 0.258924	valid's l2: 6.22724	valid's my_metric_1: 0.261979
[5]	train's l2: 5.73386	train's my_metric_1: 0.248584	valid's l2: 5.70798	valid's my_metric_1: 0.251381
[6]	train's l2: 5.26536	train's my_metric_1: 0.238853	valid's l2: 5.23758	valid's my_metric_1: 0.241379
[7]	train's l2: 4.84259	train's my_metric_1: 0.229769	valid's l2: 4.8122	valid's my_metric_1: 0.231959
[8]	train's l2: 4.46035	train's my_metric_1: 0.221225	valid's l2: 4.43004	valid's my_metric_1: 0.223261
[9]	train's l2: 4.11357	train's my_metric_1: 0.213219	valid's l2: 4.08444	valid's my_metric_1: 0.215166
[10]

...

[1]	train's l2: 8.11941	train's my_metric_1: 0.293962	valid's l2: 8.5072	valid's my_metric_1: 0.305457
Training until validation scores don't improve for 50 rounds.
[2]	train's l2: 7.42121	train's my_metric_1: 0.281395	valid's l2: 7.77991	valid's my_metric_1: 0.292594
[3]	train's l2: 6.79061	train's my_metric_1: 0.269597	valid's l2: 7.12382	valid's my_metric_1: 0.280501
[4]	train's l2: 6.22038	train's my_metric_1: 0.258536	valid's l2: 6.52814	valid's my_metric_1: 0.269112
[5]	train's l2: 5.70506	train's my_metric_1: 0.248116	valid's l2: 5.98662	valid's my_metric_1: 0.258255
[6]	train's l2: 5.23938	train's my_metric_1: 0.238367	valid's l2: 5.50118	valid's my_metric_1: 0.248185
[7]	train's l2: 4.81832	train's my_metric_1: 0.229232	valid's l2: 5.06285	valid's my_metric_1: 0.238709
[8]	train's l2: 4.43681	train's my_metric_1: 0.220691	valid's l2: 4.66418	valid's my_metric_1: 0.229818
[9]	train's l2: 4.09169	train's my_metric_1: 0.212675	valid's l2: 4.3038	valid's my_metric_1: 0.221399
[10]

[89]	train's l2: 0.541128	train's my_metric_1: 0.0813444	valid's l2: 0.636351	valid's my_metric_1: 0.0874438
[90]	train's l2: 0.53804	train's my_metric_1: 0.0811026	valid's l2: 0.634023	valid's my_metric_1: 0.0872368
[91]	train's l2: 0.535261	train's my_metric_1: 0.0808429	valid's l2: 0.631521	valid's my_metric_1: 0.0870107
[92]	train's l2: 0.532752	train's my_metric_1: 0.0806365	valid's l2: 0.630107	valid's my_metric_1: 0.0868406
[93]	train's l2: 0.52953	train's my_metric_1: 0.0803563	valid's l2: 0.627876	valid's my_metric_1: 0.0866285
[94]	train's l2: 0.527295	train's my_metric_1: 0.0801527	valid's l2: 0.62632	valid's my_metric_1: 0.086488
[95]	train's l2: 0.524564	train's my_metric_1: 0.0798825	valid's l2: 0.623504	valid's my_metric_1: 0.0862233
[96]	train's l2: 0.522109	train's my_metric_1: 0.0796596	valid's l2: 0.621446	valid's my_metric_1: 0.0860462
[97]	train's l2: 0.518871	train's my_metric_1: 0.0793347	valid's l2: 0.618872	valid's my_metric_1: 0.0858061
[98]	train's l2: 0.5160

[72]	train's l2: 0.60423	train's my_metric_1: 0.0870763	valid's l2: 0.693662	valid's my_metric_1: 0.0913471
[73]	train's l2: 0.599544	train's my_metric_1: 0.0866768	valid's l2: 0.689343	valid's my_metric_1: 0.0910115
[74]	train's l2: 0.595676	train's my_metric_1: 0.0863447	valid's l2: 0.68532	valid's my_metric_1: 0.0906962
[75]	train's l2: 0.59231	train's my_metric_1: 0.0860435	valid's l2: 0.682858	valid's my_metric_1: 0.0904622
[76]	train's l2: 0.58861	train's my_metric_1: 0.0857174	valid's l2: 0.679464	valid's my_metric_1: 0.0901882
[77]	train's l2: 0.584952	train's my_metric_1: 0.0854094	valid's l2: 0.675868	valid's my_metric_1: 0.0899023
[78]	train's l2: 0.58201	train's my_metric_1: 0.0851505	valid's l2: 0.673226	valid's my_metric_1: 0.089702
[79]	train's l2: 0.579825	train's my_metric_1: 0.0849478	valid's l2: 0.671633	valid's my_metric_1: 0.0895552
[80]	train's l2: 0.576603	train's my_metric_1: 0.0846277	valid's l2: 0.668316	valid's my_metric_1: 0.0892809
[81]	train's l2: 0.572613

[51]	train's l2: 0.733323	train's my_metric_1: 0.097582	valid's l2: 0.778863	valid's my_metric_1: 0.101084
[52]	train's l2: 0.724646	train's my_metric_1: 0.0969341	valid's l2: 0.770429	valid's my_metric_1: 0.100452
[53]	train's l2: 0.716073	train's my_metric_1: 0.0962594	valid's l2: 0.761781	valid's my_metric_1: 0.0997729
[54]	train's l2: 0.70788	train's my_metric_1: 0.0956192	valid's l2: 0.754578	valid's my_metric_1: 0.0991876
[55]	train's l2: 0.699657	train's my_metric_1: 0.0949973	valid's l2: 0.748	valid's my_metric_1: 0.0986739
[56]	train's l2: 0.692122	train's my_metric_1: 0.094401	valid's l2: 0.740965	valid's my_metric_1: 0.0981246
[57]	train's l2: 0.684942	train's my_metric_1: 0.09383	valid's l2: 0.734997	valid's my_metric_1: 0.0976258
[58]	train's l2: 0.678307	train's my_metric_1: 0.0932864	valid's l2: 0.728546	valid's my_metric_1: 0.0971081
[59]	train's l2: 0.670864	train's my_metric_1: 0.0926534	valid's l2: 0.72121	valid's my_metric_1: 0.0965035
[60]	train's l2: 0.664372	trai

[26]	train's l2: 1.38984	train's my_metric_1: 0.132746	valid's l2: 1.46389	valid's my_metric_1: 0.136014
[27]	train's l2: 1.32851	train's my_metric_1: 0.130106	valid's l2: 1.40424	valid's my_metric_1: 0.133496
[28]	train's l2: 1.27252	train's my_metric_1: 0.127676	valid's l2: 1.348	valid's my_metric_1: 0.131112
[29]	train's l2: 1.22208	train's my_metric_1: 0.125374	valid's l2: 1.29984	valid's my_metric_1: 0.128932
[30]	train's l2: 1.17595	train's my_metric_1: 0.123261	valid's l2: 1.25526	valid's my_metric_1: 0.12692
[31]	train's l2: 1.13357	train's my_metric_1: 0.121269	valid's l2: 1.21376	valid's my_metric_1: 0.125021
[32]	train's l2: 1.09469	train's my_metric_1: 0.119341	valid's l2: 1.17572	valid's my_metric_1: 0.12317
[33]	train's l2: 1.05893	train's my_metric_1: 0.117546	valid's l2: 1.14097	valid's my_metric_1: 0.121461
[34]	train's l2: 1.02603	train's my_metric_1: 0.11579	valid's l2: 1.10962	valid's my_metric_1: 0.119831
[35]	train's l2: 0.996395	train's my_metric_1: 0.114162	vali

In [59]:
train_features = X_train_1.columns
model_1.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_1.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
6,时间戳,430
40,湿度日中值,401
36,辐照度日中值,347
37,风速日中值,294
32,day,293
51,辐照度月平均,267
38,温度日中值,258
54,压强月平均,228
39,压强日中值,204
25,辐照度+3,162


In [60]:
X_train_2 = data_2[:len(train_2)].drop(['时间','实发辐照度','实际功率','id'],axis=1)

y_train_2 = data_2[:len(train_2)]['实际功率']

X_test_2 = data_2[len(train_2):].drop(['时间','实发辐照度','实际功率','id'],axis=1)

In [61]:
from sklearn.metrics import mean_absolute_error
def my_metric_2(y_true,y_pred): 
    y_true_2 = y_true[y_true>=10*0.03]
    y_pred_2 = y_pred[y_true>=10*0.03]
    score = mean_absolute_error(y_true_2,y_pred_2)/10.0
    return 'my_metric_2',score,False  # False表示score不是越大越好(is_bigger_better)

In [62]:
prediction_2 = test_2[['id']]

X_train_2_loc = X_train_2.values
X_test_2_loc = X_test_2.values
y_train_2_loc = y_train_2.values

model_2 = lgb.LGBMRegressor('gbdt',num_leaves=41,max_depth=-1,learning_rate=0.05,n_estimators=200,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.01,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_train_2_loc, n_folds=10, shuffle=True, random_state=2018))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model_2.fit(X_train_2_loc[train_index], y_train_2_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_2_loc[train_index], y_train_2_loc[train_index]), 
                                    (X_train_2_loc[test_index], y_train_2_loc[test_index])],
                          eval_metric=my_metric_2,
                          early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_2'])
    loss += lgb_model.best_score_['valid']['my_metric_2']
    test_pred= lgb_model.predict(X_test_2_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_2['predict_%s' % str(i)] = test_pred
print('mae:', baseloss, loss/10)

p = prediction_2.drop(['id'],axis=1)
prediction_2['prediction']=p.mean(axis=1)

d:\Users\onLine\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Fold 0
[1]	train's l2: 7.3809	train's my_metric_2: 0.294941	valid's l2: 7.71639	valid's my_metric_2: 0.293761
Training until validation scores don't improve for 100 rounds.
[2]	train's l2: 6.78197	train's my_metric_2: 0.283359	valid's l2: 7.09559	valid's my_metric_2: 0.28231
[3]	train's l2: 6.2407	train's my_metric_2: 0.272502	valid's l2: 6.53864	valid's my_metric_2: 0.271758
[4]	train's l2: 5.75074	train's my_metric_2: 0.262387	valid's l2: 6.03614	valid's my_metric_2: 0.261997
[5]	train's l2: 5.30866	train's my_metric_2: 0.252949	valid's l2: 5.5778	valid's my_metric_2: 0.252737
[6]	train's l2: 4.90649	train's my_metric_2: 0.243965	valid's l2: 5.16027	valid's my_metric_2: 0.243976
[7]	train's l2: 4.54146	train's my_metric_2: 0.235425	valid's l2: 4.78487	valid's my_metric_2: 0.235746
[8]	train's l2: 4.2122	train's my_metric_2: 0.227558	valid's l2: 4.4452	valid's my_metric_2: 0.228142
[9]	train's l2: 3.91497	train's my_metric_2: 0.220214	valid's l2: 4.13965	valid's my_metric_2: 0.221062


[81]	train's l2: 0.685445	train's my_metric_2: 0.0957285	valid's l2: 0.865787	valid's my_metric_2: 0.102793
[82]	train's l2: 0.682499	train's my_metric_2: 0.0954829	valid's l2: 0.863381	valid's my_metric_2: 0.102592
[83]	train's l2: 0.677812	train's my_metric_2: 0.0950523	valid's l2: 0.858716	valid's my_metric_2: 0.102226
[84]	train's l2: 0.674925	train's my_metric_2: 0.09481	valid's l2: 0.856501	valid's my_metric_2: 0.102042
[85]	train's l2: 0.670534	train's my_metric_2: 0.0944125	valid's l2: 0.851918	valid's my_metric_2: 0.101693
[86]	train's l2: 0.667478	train's my_metric_2: 0.0941733	valid's l2: 0.849655	valid's my_metric_2: 0.101539
[87]	train's l2: 0.662594	train's my_metric_2: 0.0937306	valid's l2: 0.844698	valid's my_metric_2: 0.101175
[88]	train's l2: 0.659285	train's my_metric_2: 0.0934451	valid's l2: 0.841842	valid's my_metric_2: 0.100938
[89]	train's l2: 0.654662	train's my_metric_2: 0.0930267	valid's l2: 0.837608	valid's my_metric_2: 0.100648
[90]	train's l2: 0.651403	trai

[158]	train's l2: 0.520813	train's my_metric_2: 0.0819304	valid's l2: 0.734725	valid's my_metric_2: 0.0929746
[159]	train's l2: 0.519698	train's my_metric_2: 0.0818274	valid's l2: 0.734223	valid's my_metric_2: 0.0929388
[160]	train's l2: 0.518368	train's my_metric_2: 0.0817011	valid's l2: 0.73276	valid's my_metric_2: 0.0928257
[161]	train's l2: 0.517336	train's my_metric_2: 0.0816163	valid's l2: 0.73197	valid's my_metric_2: 0.0927699
[162]	train's l2: 0.515466	train's my_metric_2: 0.0814575	valid's l2: 0.730868	valid's my_metric_2: 0.0926676
[163]	train's l2: 0.514572	train's my_metric_2: 0.0813722	valid's l2: 0.730513	valid's my_metric_2: 0.0926282
[164]	train's l2: 0.513422	train's my_metric_2: 0.081264	valid's l2: 0.729318	valid's my_metric_2: 0.0925384
[165]	train's l2: 0.512271	train's my_metric_2: 0.0811649	valid's l2: 0.72872	valid's my_metric_2: 0.0924931
[166]	train's l2: 0.511243	train's my_metric_2: 0.0810787	valid's l2: 0.72851	valid's my_metric_2: 0.0924691
[167]	train's l

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

[37]	train's l2: 1.11621	train's my_metric_2: 0.125831	valid's l2: 1.11247	valid's my_metric_2: 0.128121
[38]	train's l2: 1.09173	train's my_metric_2: 0.124368	valid's l2: 1.08906	valid's my_metric_2: 0.126686
[39]	train's l2: 1.0703	train's my_metric_2: 0.123106	valid's l2: 1.06988	valid's my_metric_2: 0.125523
[40]	train's l2: 1.04857	train's my_metric_2: 0.121754	valid's l2: 1.05032	valid's my_metric_2: 0.124281
[41]	train's l2: 1.02848	train's my_metric_2: 0.120457	valid's l2: 1.03207	valid's my_metric_2: 0.12313
[42]	train's l2: 1.01114	train's my_metric_2: 0.119398	valid's l2: 1.0169	valid's my_metric_2: 0.122245
[43]	train's l2: 0.994122	train's my_metric_2: 0.118307	valid's l2: 1.0012	valid's my_metric_2: 0.121206
[44]	train's l2: 0.978761	train's my_metric_2: 0.117266	valid's l2: 0.986057	valid's my_metric_2: 0.120168
[45]	train's l2: 0.962655	train's my_metric_2: 0.116153	valid's l2: 0.97153	valid's my_metric_2: 0.119148
[46]	train's l2: 0.949363	train's my_metric_2: 0.115228

[118]	train's l2: 0.602493	train's my_metric_2: 0.0876202	valid's l2: 0.669123	valid's my_metric_2: 0.0949626
[119]	train's l2: 0.600101	train's my_metric_2: 0.0874221	valid's l2: 0.667251	valid's my_metric_2: 0.0947896
[120]	train's l2: 0.598608	train's my_metric_2: 0.0872993	valid's l2: 0.666965	valid's my_metric_2: 0.0947604
[121]	train's l2: 0.596995	train's my_metric_2: 0.087159	valid's l2: 0.666061	valid's my_metric_2: 0.0946964
[122]	train's l2: 0.595508	train's my_metric_2: 0.087035	valid's l2: 0.665439	valid's my_metric_2: 0.0946161
[123]	train's l2: 0.594176	train's my_metric_2: 0.0869244	valid's l2: 0.664996	valid's my_metric_2: 0.0945915
[124]	train's l2: 0.592891	train's my_metric_2: 0.0868166	valid's l2: 0.664506	valid's my_metric_2: 0.0945495
[125]	train's l2: 0.591012	train's my_metric_2: 0.0866736	valid's l2: 0.66256	valid's my_metric_2: 0.0944008
[126]	train's l2: 0.5897	train's my_metric_2: 0.0865699	valid's l2: 0.662462	valid's my_metric_2: 0.0943683
[127]	train's l

[200]	train's l2: 0.487708	train's my_metric_2: 0.0779456	valid's l2: 0.601408	valid's my_metric_2: 0.0889368
Did not meet early stopping. Best iteration is:
[200]	train's l2: 0.487708	train's my_metric_2: 0.0779456	valid's l2: 0.601408	valid's my_metric_2: 0.0889368
Fold 9
[1]	train's l2: 7.45127	train's my_metric_2: 0.293273	valid's l2: 7.1137	valid's my_metric_2: 0.291897
Training until validation scores don't improve for 100 rounds.
[2]	train's l2: 6.84977	train's my_metric_2: 0.281861	valid's l2: 6.53815	valid's my_metric_2: 0.280705
[3]	train's l2: 6.30167	train's my_metric_2: 0.271089	valid's l2: 6.01504	valid's my_metric_2: 0.270193
[4]	train's l2: 5.81078	train's my_metric_2: 0.261155	valid's l2: 5.54589	valid's my_metric_2: 0.260402
[5]	train's l2: 5.36559	train's my_metric_2: 0.251776	valid's l2: 5.12001	valid's my_metric_2: 0.251198
[6]	train's l2: 4.96359	train's my_metric_2: 0.243026	valid's l2: 4.73625	valid's my_metric_2: 0.242627
[7]	train's l2: 4.59388	train's my_metr

[80]	train's l2: 0.708233	train's my_metric_2: 0.0964725	valid's l2: 0.737689	valid's my_metric_2: 0.100534
[81]	train's l2: 0.705317	train's my_metric_2: 0.0962304	valid's l2: 0.736103	valid's my_metric_2: 0.100335
[82]	train's l2: 0.702645	train's my_metric_2: 0.0959963	valid's l2: 0.735192	valid's my_metric_2: 0.100169
[83]	train's l2: 0.697635	train's my_metric_2: 0.0955671	valid's l2: 0.731468	valid's my_metric_2: 0.0998161
[84]	train's l2: 0.691884	train's my_metric_2: 0.0950748	valid's l2: 0.726517	valid's my_metric_2: 0.0994144
[85]	train's l2: 0.688598	train's my_metric_2: 0.0948312	valid's l2: 0.724126	valid's my_metric_2: 0.0992114
[86]	train's l2: 0.683712	train's my_metric_2: 0.0944062	valid's l2: 0.719857	valid's my_metric_2: 0.0988249
[87]	train's l2: 0.680684	train's my_metric_2: 0.0941607	valid's l2: 0.717448	valid's my_metric_2: 0.0985885
[88]	train's l2: 0.676869	train's my_metric_2: 0.0938758	valid's l2: 0.713468	valid's my_metric_2: 0.0983224
[89]	train's l2: 0.672

[163]	train's l2: 0.532608	train's my_metric_2: 0.0820956	valid's l2: 0.621062	valid's my_metric_2: 0.0905556
[164]	train's l2: 0.531449	train's my_metric_2: 0.0819907	valid's l2: 0.620801	valid's my_metric_2: 0.0905317
[165]	train's l2: 0.530487	train's my_metric_2: 0.0819088	valid's l2: 0.620343	valid's my_metric_2: 0.0905012
[166]	train's l2: 0.528996	train's my_metric_2: 0.0817966	valid's l2: 0.619034	valid's my_metric_2: 0.0904072
[167]	train's l2: 0.527436	train's my_metric_2: 0.0816626	valid's l2: 0.617246	valid's my_metric_2: 0.0902888
[168]	train's l2: 0.526067	train's my_metric_2: 0.0815449	valid's l2: 0.616697	valid's my_metric_2: 0.0902357
[169]	train's l2: 0.52503	train's my_metric_2: 0.081464	valid's l2: 0.615873	valid's my_metric_2: 0.0901625
[170]	train's l2: 0.523969	train's my_metric_2: 0.0813754	valid's l2: 0.615298	valid's my_metric_2: 0.090121
[171]	train's l2: 0.523132	train's my_metric_2: 0.0813061	valid's l2: 0.615361	valid's my_metric_2: 0.0901373
[172]	train's

In [63]:
train_features = X_train_2.columns
model_2.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_2.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
6,时间戳,1529
13,辐照度+1,586
27,压强月平均,475
26,温度月平均,434
9,风向-1,346
18,湿度+1,332
28,湿度月平均,331
10,温度-1,317
15,风向+1,317
25,风速月平均,289


In [64]:
from sklearn.metrics import mean_absolute_error
def my_metric_3(y_true,y_pred): 
    y_true_3 = y_true[y_true>=40*0.03]
    y_pred_3 = y_pred[y_true>=40*0.03]
    score = mean_absolute_error(y_true_3,y_pred_3)/40.0
    return 'my_metric_3',score,False  # False表示score不是越大越好(is_bigger_better)

In [65]:
X_train_3 = data_3[:len(train_3)].drop(['时间','实发辐照度','实际功率','id'],axis=1)

y_train_3 = data_3[:len(train_3)]['实际功率']

X_test_3 = data_3[len(train_3):].drop(['时间','实发辐照度','实际功率','id'],axis=1)


prediction_3 = test_3[['id']]

X_train_3_loc = X_train_3.values
X_test_3_loc = X_test_3.values
y_train_3_loc = y_train_3.values

model_3 = lgb.LGBMRegressor('gbdt',num_leaves=51,max_depth=-1,learning_rate=0.05,n_estimators=200,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.01,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_train_3_loc, n_folds=10, shuffle=True, random_state=2018))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model_3.fit(X_train_3_loc[train_index], y_train_3_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_3_loc[train_index], y_train_3_loc[train_index]), 
                                    (X_train_3_loc[test_index], y_train_3_loc[test_index])],
                          eval_metric=my_metric_3,
                          early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_3'])
    loss += lgb_model.best_score_['valid']['my_metric_3']
    test_pred= lgb_model.predict(X_test_3_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_3['predict_%s' % str(i)] = test_pred
print('mae:', baseloss, loss/10)

p = prediction_3.drop(['id'],axis=1)
prediction_3['prediction']=p.mean(axis=1)

d:\Users\onLine\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Fold 0
[1]	train's l2: 109.162	train's my_metric_3: 0.273648	valid's l2: 113.189	valid's my_metric_3: 0.278067
Training until validation scores don't improve for 100 rounds.
[2]	train's l2: 100.06	train's my_metric_3: 0.262361	valid's l2: 103.785	valid's my_metric_3: 0.266689
[3]	train's l2: 91.7965	train's my_metric_3: 0.251749	valid's l2: 95.2501	valid's my_metric_3: 0.256018
[4]	train's l2: 84.3371	train's my_metric_3: 0.241741	valid's l2: 87.5493	valid's my_metric_3: 0.246011
[5]	train's l2: 77.5914	train's my_metric_3: 0.232402	valid's l2: 80.6067	valid's my_metric_3: 0.236721
[6]	train's l2: 71.4818	train's my_metric_3: 0.223651	valid's l2: 74.2979	valid's my_metric_3: 0.227934
[7]	train's l2: 65.9273	train's my_metric_3: 0.21528	valid's l2: 68.5483	valid's my_metric_3: 0.219556
[8]	train's l2: 60.913	train's my_metric_3: 0.207521	valid's l2: 63.3801	valid's my_metric_3: 0.211761
[9]	train's l2: 56.3507	train's my_metric_3: 0.200214	valid's l2: 58.6882	valid's my_metric_3: 0.2044

[92]	train's l2: 6.00974	train's my_metric_3: 0.0657038	valid's l2: 7.85522	valid's my_metric_3: 0.0731907
[93]	train's l2: 5.95304	train's my_metric_3: 0.0653999	valid's l2: 7.80762	valid's my_metric_3: 0.072927
[94]	train's l2: 5.89591	train's my_metric_3: 0.0650719	valid's l2: 7.74775	valid's my_metric_3: 0.0726186
[95]	train's l2: 5.84636	train's my_metric_3: 0.0647576	valid's l2: 7.70195	valid's my_metric_3: 0.0723877
[96]	train's l2: 5.79201	train's my_metric_3: 0.0644481	valid's l2: 7.6404	valid's my_metric_3: 0.0720832
[97]	train's l2: 5.74653	train's my_metric_3: 0.064175	valid's l2: 7.59833	valid's my_metric_3: 0.071863
[98]	train's l2: 5.70324	train's my_metric_3: 0.0639206	valid's l2: 7.55868	valid's my_metric_3: 0.0716371
[99]	train's l2: 5.66778	train's my_metric_3: 0.0637089	valid's l2: 7.5296	valid's my_metric_3: 0.0714409
[100]	train's l2: 5.61061	train's my_metric_3: 0.0633888	valid's l2: 7.46846	valid's my_metric_3: 0.0711814
[101]	train's l2: 5.56418	train's my_metr

[180]	train's l2: 3.7504	train's my_metric_3: 0.0509564	valid's l2: 5.90751	valid's my_metric_3: 0.0617329
[181]	train's l2: 3.73166	train's my_metric_3: 0.050826	valid's l2: 5.8796	valid's my_metric_3: 0.0616095
[182]	train's l2: 3.72163	train's my_metric_3: 0.0507485	valid's l2: 5.8739	valid's my_metric_3: 0.0615689
[183]	train's l2: 3.7084	train's my_metric_3: 0.0506466	valid's l2: 5.86076	valid's my_metric_3: 0.0615043
[184]	train's l2: 3.69269	train's my_metric_3: 0.050537	valid's l2: 5.85152	valid's my_metric_3: 0.0614301
[185]	train's l2: 3.68003	train's my_metric_3: 0.05045	valid's l2: 5.83916	valid's my_metric_3: 0.0613734
[186]	train's l2: 3.66699	train's my_metric_3: 0.0503448	valid's l2: 5.82933	valid's my_metric_3: 0.0613153
[187]	train's l2: 3.65385	train's my_metric_3: 0.0502219	valid's l2: 5.81264	valid's my_metric_3: 0.0611915
[188]	train's l2: 3.64411	train's my_metric_3: 0.0501399	valid's l2: 5.80425	valid's my_metric_3: 0.0611299
[189]	train's l2: 3.63566	train's my

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

[30]	train's l2: 16.917	train's my_metric_3: 0.115595	valid's l2: 17.6761	valid's my_metric_3: 0.113777
[31]	train's l2: 16.3129	train's my_metric_3: 0.113578	valid's l2: 17.0743	valid's my_metric_3: 0.11176
[32]	train's l2: 15.7364	train's my_metric_3: 0.111524	valid's l2: 16.5125	valid's my_metric_3: 0.109785
[33]	train's l2: 15.228	train's my_metric_3: 0.109755	valid's l2: 16.0279	valid's my_metric_3: 0.10816
[34]	train's l2: 14.7627	train's my_metric_3: 0.10809	valid's l2: 15.5664	valid's my_metric_3: 0.106486
[35]	train's l2: 14.2971	train's my_metric_3: 0.10639	valid's l2: 15.12	valid's my_metric_3: 0.104812
[36]	train's l2: 13.8814	train's my_metric_3: 0.104767	valid's l2: 14.7196	valid's my_metric_3: 0.10327
[37]	train's l2: 13.5043	train's my_metric_3: 0.103293	valid's l2: 14.3681	valid's my_metric_3: 0.10181
[38]	train's l2: 13.1381	train's my_metric_3: 0.101897	valid's l2: 14.019	valid's my_metric_3: 0.100522
[39]	train's l2: 12.7615	train's my_metric_3: 0.100421	valid's l2:

[114]	train's l2: 5.09329	train's my_metric_3: 0.0601999	valid's l2: 6.4572	valid's my_metric_3: 0.0652514
[115]	train's l2: 5.05733	train's my_metric_3: 0.0599928	valid's l2: 6.42715	valid's my_metric_3: 0.065116
[116]	train's l2: 5.02976	train's my_metric_3: 0.0597869	valid's l2: 6.40278	valid's my_metric_3: 0.0649566
[117]	train's l2: 5.007	train's my_metric_3: 0.0596203	valid's l2: 6.38141	valid's my_metric_3: 0.064834
[118]	train's l2: 4.98088	train's my_metric_3: 0.0594372	valid's l2: 6.35781	valid's my_metric_3: 0.0647157
[119]	train's l2: 4.95428	train's my_metric_3: 0.0592714	valid's l2: 6.32863	valid's my_metric_3: 0.0645906
[120]	train's l2: 4.91698	train's my_metric_3: 0.0590234	valid's l2: 6.29681	valid's my_metric_3: 0.0643977
[121]	train's l2: 4.88909	train's my_metric_3: 0.0588225	valid's l2: 6.26317	valid's my_metric_3: 0.0641789
[122]	train's l2: 4.85741	train's my_metric_3: 0.0586509	valid's l2: 6.24427	valid's my_metric_3: 0.0641185
[123]	train's l2: 4.83449	train's

[1]	train's l2: 109.1	train's my_metric_3: 0.274218	valid's l2: 114.659	valid's my_metric_3: 0.277856
Training until validation scores don't improve for 100 rounds.
[2]	train's l2: 100.001	train's my_metric_3: 0.262936	valid's l2: 105.209	valid's my_metric_3: 0.26662
[3]	train's l2: 91.723	train's my_metric_3: 0.252252	valid's l2: 96.6979	valid's my_metric_3: 0.256185
[4]	train's l2: 84.2771	train's my_metric_3: 0.242282	valid's l2: 88.9571	valid's my_metric_3: 0.246271
[5]	train's l2: 77.5127	train's my_metric_3: 0.232919	valid's l2: 81.9962	valid's my_metric_3: 0.237088
[6]	train's l2: 71.3814	train's my_metric_3: 0.224062	valid's l2: 75.6716	valid's my_metric_3: 0.228224
[7]	train's l2: 65.8398	train's my_metric_3: 0.215773	valid's l2: 69.9437	valid's my_metric_3: 0.220026
[8]	train's l2: 60.8133	train's my_metric_3: 0.207864	valid's l2: 64.7783	valid's my_metric_3: 0.212127
[9]	train's l2: 56.2267	train's my_metric_3: 0.200469	valid's l2: 60.1043	valid's my_metric_3: 0.204972
[10]	

[88]	train's l2: 6.07282	train's my_metric_3: 0.0663683	valid's l2: 9.28095	valid's my_metric_3: 0.0781338
[89]	train's l2: 6.02411	train's my_metric_3: 0.0660618	valid's l2: 9.23646	valid's my_metric_3: 0.0779259
[90]	train's l2: 5.96942	train's my_metric_3: 0.0657068	valid's l2: 9.17214	valid's my_metric_3: 0.0775764
[91]	train's l2: 5.89972	train's my_metric_3: 0.0653202	valid's l2: 9.09088	valid's my_metric_3: 0.0771953
[92]	train's l2: 5.84561	train's my_metric_3: 0.0650377	valid's l2: 9.03138	valid's my_metric_3: 0.0769615
[93]	train's l2: 5.8095	train's my_metric_3: 0.0647797	valid's l2: 8.9916	valid's my_metric_3: 0.0767418
[94]	train's l2: 5.75107	train's my_metric_3: 0.0644451	valid's l2: 8.94383	valid's my_metric_3: 0.0764693
[95]	train's l2: 5.7078	train's my_metric_3: 0.0641466	valid's l2: 8.89988	valid's my_metric_3: 0.0762487
[96]	train's l2: 5.65737	train's my_metric_3: 0.0638828	valid's l2: 8.84043	valid's my_metric_3: 0.0759774
[97]	train's l2: 5.61279	train's my_metr

[173]	train's l2: 3.80167	train's my_metric_3: 0.0513942	valid's l2: 6.96261	valid's my_metric_3: 0.0661209
[174]	train's l2: 3.78426	train's my_metric_3: 0.0512705	valid's l2: 6.94594	valid's my_metric_3: 0.0660217
[175]	train's l2: 3.76712	train's my_metric_3: 0.0511553	valid's l2: 6.92537	valid's my_metric_3: 0.06594
[176]	train's l2: 3.75648	train's my_metric_3: 0.0510578	valid's l2: 6.91314	valid's my_metric_3: 0.0658553
[177]	train's l2: 3.74102	train's my_metric_3: 0.0509113	valid's l2: 6.90416	valid's my_metric_3: 0.0657755
[178]	train's l2: 3.72289	train's my_metric_3: 0.0507735	valid's l2: 6.8875	valid's my_metric_3: 0.0656965
[179]	train's l2: 3.71115	train's my_metric_3: 0.0506736	valid's l2: 6.87552	valid's my_metric_3: 0.0656266
[180]	train's l2: 3.70055	train's my_metric_3: 0.0505904	valid's l2: 6.86192	valid's my_metric_3: 0.0655451
[181]	train's l2: 3.68693	train's my_metric_3: 0.0504943	valid's l2: 6.8489	valid's my_metric_3: 0.0654647
[182]	train's l2: 3.67589	train'

In [66]:
train_features = X_train_3.columns
model_3.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_3.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
6,时间戳,878
9,温度日最小,578
2,风向,520
8,风速日最小,496
11,湿度日最小,492
28,风速日平均,456
31,湿度日平均,432
30,压强日平均,417
35,压强月平均,408
34,温度月平均,390


In [67]:
from sklearn.metrics import mean_absolute_error
def my_metric_4(y_true,y_pred): 
    y_true_4 = y_true[y_true>=50*0.03]
    y_pred_4 = y_pred[y_true>=50*0.03]
    score = mean_absolute_error(y_true_4,y_pred_4)/50.0
    return 'my_metric_4',score,False  # False表示score不是越大越好(is_bigger_better)

In [68]:
X_train_4 = data_4[:len(train_4)].drop(['时间','实发辐照度','实际功率','id'],axis=1)

y_train_4 = data_4[:len(train_4)]['实际功率']

X_test_4 = data_4[len(train_4):].drop(['时间','实发辐照度','实际功率','id'],axis=1)


prediction_4 = test_4[['id']]

X_train_4_loc = X_train_4.values
X_test_4_loc = X_test_4.values
y_train_4_loc = y_train_4.values

model_4 = lgb.LGBMRegressor('gbdt',num_leaves=51,max_depth=-1,learning_rate=0.05,n_estimators=200,max_bin=255,subsample_for_bin=200000,
                         objective='regression',min_split_gain=0,min_child_weight=0.01,min_child_samples=20,subsample=1,subsample_freq=1,
                         colsample_bytree=1, reg_alpha=0, reg_lambda=0, random_state=2018,n_jobs=-1)
# 五折交叉训练，构造五个模型
skf=list(StratifiedKFold(y_train_4_loc, n_folds=10, shuffle=True, random_state=2018))
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf):
    print("Fold", i)
    lgb_model = model_4.fit(X_train_4_loc[train_index], y_train_4_loc[train_index],
                          eval_names =['train','valid'],
                          eval_set=[(X_train_4_loc[train_index], y_train_4_loc[train_index]), 
                                    (X_train_4_loc[test_index], y_train_4_loc[test_index])],
                          eval_metric=my_metric_4,
                          early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['my_metric_4'])
    loss += lgb_model.best_score_['valid']['my_metric_4']
    test_pred= lgb_model.predict(X_test_4_loc, num_iteration=lgb_model.best_iteration_)
#     print('test mean:', test_pred.mean())
    prediction_4['predict_%s' % str(i)] = test_pred
print('mae:', baseloss, loss/10)

p = prediction_4.drop(['id'],axis=1)
prediction_4['prediction']=p.mean(axis=1)

d:\Users\onLine\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Fold 0
[1]	train's l2: 206.117	train's my_metric_4: 0.29824	valid's l2: 206.551	valid's my_metric_4: 0.294886
Training until validation scores don't improve for 100 rounds.
[2]	train's l2: 188.504	train's my_metric_4: 0.285477	valid's l2: 188.97	valid's my_metric_4: 0.282335
[3]	train's l2: 172.56	train's my_metric_4: 0.273521	valid's l2: 173.156	valid's my_metric_4: 0.270646
[4]	train's l2: 158.164	train's my_metric_4: 0.26226	valid's l2: 158.899	valid's my_metric_4: 0.259694
[5]	train's l2: 145.138	train's my_metric_4: 0.251728	valid's l2: 146.038	valid's my_metric_4: 0.24947
[6]	train's l2: 133.368	train's my_metric_4: 0.241796	valid's l2: 134.362	valid's my_metric_4: 0.239726
[7]	train's l2: 122.719	train's my_metric_4: 0.232499	valid's l2: 123.811	valid's my_metric_4: 0.2306
[8]	train's l2: 113.037	train's my_metric_4: 0.22372	valid's l2: 114.266	valid's my_metric_4: 0.222083
[9]	train's l2: 104.213	train's my_metric_4: 0.215396	valid's l2: 105.571	valid's my_metric_4: 0.213972
[1

[93]	train's l2: 10.8937	train's my_metric_4: 0.0737045	valid's l2: 14.0758	valid's my_metric_4: 0.0816171
[94]	train's l2: 10.8195	train's my_metric_4: 0.0734019	valid's l2: 13.9939	valid's my_metric_4: 0.0813443
[95]	train's l2: 10.751	train's my_metric_4: 0.0731201	valid's l2: 13.9221	valid's my_metric_4: 0.0810757
[96]	train's l2: 10.6738	train's my_metric_4: 0.0728231	valid's l2: 13.8491	valid's my_metric_4: 0.0808302
[97]	train's l2: 10.6134	train's my_metric_4: 0.072591	valid's l2: 13.7931	valid's my_metric_4: 0.0806101
[98]	train's l2: 10.5497	train's my_metric_4: 0.0723304	valid's l2: 13.7365	valid's my_metric_4: 0.0803765
[99]	train's l2: 10.4949	train's my_metric_4: 0.0721273	valid's l2: 13.6894	valid's my_metric_4: 0.0801973
[100]	train's l2: 10.4399	train's my_metric_4: 0.0719086	valid's l2: 13.6515	valid's my_metric_4: 0.0800419
[101]	train's l2: 10.3644	train's my_metric_4: 0.0715999	valid's l2: 13.5752	valid's my_metric_4: 0.0797714
[102]	train's l2: 10.318	train's my_m

[178]	train's l2: 7.83573	train's my_metric_4: 0.0611712	valid's l2: 11.4224	valid's my_metric_4: 0.0719498
[179]	train's l2: 7.82238	train's my_metric_4: 0.0611185	valid's l2: 11.4212	valid's my_metric_4: 0.071953
[180]	train's l2: 7.79575	train's my_metric_4: 0.0610161	valid's l2: 11.4141	valid's my_metric_4: 0.0719092
[181]	train's l2: 7.78173	train's my_metric_4: 0.0609526	valid's l2: 11.4009	valid's my_metric_4: 0.0718619
[182]	train's l2: 7.75327	train's my_metric_4: 0.0608113	valid's l2: 11.3736	valid's my_metric_4: 0.0717615
[183]	train's l2: 7.73313	train's my_metric_4: 0.0607161	valid's l2: 11.3508	valid's my_metric_4: 0.0716686
[184]	train's l2: 7.71465	train's my_metric_4: 0.060641	valid's l2: 11.3508	valid's my_metric_4: 0.0716677
[185]	train's l2: 7.69885	train's my_metric_4: 0.0605764	valid's l2: 11.341	valid's my_metric_4: 0.071625
[186]	train's l2: 7.68177	train's my_metric_4: 0.0604765	valid's l2: 11.3249	valid's my_metric_4: 0.0715358
[187]	train's l2: 7.66945	train'

d:\Users\onLine\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

[41]	train's l2: 20.4338	train's my_metric_4: 0.104485	valid's l2: 22.0512	valid's my_metric_4: 0.109988
[42]	train's l2: 19.9627	train's my_metric_4: 0.103272	valid's l2: 21.6296	valid's my_metric_4: 0.108978
[43]	train's l2: 19.5236	train's my_metric_4: 0.102118	valid's l2: 21.2025	valid's my_metric_4: 0.107899
[44]	train's l2: 19.1067	train's my_metric_4: 0.101013	valid's l2: 20.8199	valid's my_metric_4: 0.10684
[45]	train's l2: 18.7039	train's my_metric_4: 0.0999009	valid's l2: 20.4643	valid's my_metric_4: 0.105825
[46]	train's l2: 18.311	train's my_metric_4: 0.0988066	valid's l2: 20.088	valid's my_metric_4: 0.104797
[47]	train's l2: 17.9391	train's my_metric_4: 0.0977238	valid's l2: 19.7485	valid's my_metric_4: 0.103809
[48]	train's l2: 17.6103	train's my_metric_4: 0.0967749	valid's l2: 19.4645	valid's my_metric_4: 0.102957
[49]	train's l2: 17.3167	train's my_metric_4: 0.0958678	valid's l2: 19.183	valid's my_metric_4: 0.102071
[50]	train's l2: 17.0248	train's my_metric_4: 0.094975

[127]	train's l2: 9.40705	train's my_metric_4: 0.0674484	valid's l2: 12.2962	valid's my_metric_4: 0.0782532
[128]	train's l2: 9.37318	train's my_metric_4: 0.0673044	valid's l2: 12.2751	valid's my_metric_4: 0.0781904
[129]	train's l2: 9.32992	train's my_metric_4: 0.0671075	valid's l2: 12.2439	valid's my_metric_4: 0.0780657
[130]	train's l2: 9.29562	train's my_metric_4: 0.0669791	valid's l2: 12.219	valid's my_metric_4: 0.0779852
[131]	train's l2: 9.26163	train's my_metric_4: 0.066843	valid's l2: 12.1992	valid's my_metric_4: 0.0779003
[132]	train's l2: 9.22716	train's my_metric_4: 0.0667033	valid's l2: 12.1713	valid's my_metric_4: 0.0777964
[133]	train's l2: 9.19807	train's my_metric_4: 0.0665841	valid's l2: 12.1343	valid's my_metric_4: 0.077679
[134]	train's l2: 9.1713	train's my_metric_4: 0.0664608	valid's l2: 12.1126	valid's my_metric_4: 0.0775784
[135]	train's l2: 9.1348	train's my_metric_4: 0.0663016	valid's l2: 12.0807	valid's my_metric_4: 0.0774731
[136]	train's l2: 9.09183	train's

[21]	train's l2: 45.4019	train's my_metric_4: 0.149185	valid's l2: 46.6696	valid's my_metric_4: 0.149648
[22]	train's l2: 42.8839	train's my_metric_4: 0.145538	valid's l2: 44.1491	valid's my_metric_4: 0.146111
[23]	train's l2: 40.5536	train's my_metric_4: 0.142093	valid's l2: 41.8352	valid's my_metric_4: 0.142883
[24]	train's l2: 38.4586	train's my_metric_4: 0.138895	valid's l2: 39.7446	valid's my_metric_4: 0.139849
[25]	train's l2: 36.5361	train's my_metric_4: 0.135848	valid's l2: 37.8377	valid's my_metric_4: 0.136969
[26]	train's l2: 34.7711	train's my_metric_4: 0.132978	valid's l2: 36.0714	valid's my_metric_4: 0.134285
[27]	train's l2: 33.1845	train's my_metric_4: 0.130298	valid's l2: 34.498	valid's my_metric_4: 0.131877
[28]	train's l2: 31.7346	train's my_metric_4: 0.127806	valid's l2: 33.0573	valid's my_metric_4: 0.129592
[29]	train's l2: 30.3822	train's my_metric_4: 0.125428	valid's l2: 31.6886	valid's my_metric_4: 0.127336
[30]	train's l2: 29.1299	train's my_metric_4: 0.123132	v

[109]	train's l2: 10.074	train's my_metric_4: 0.0702409	valid's l2: 12.2014	valid's my_metric_4: 0.0780149
[110]	train's l2: 10.0212	train's my_metric_4: 0.0700332	valid's l2: 12.1569	valid's my_metric_4: 0.0778152
[111]	train's l2: 9.9684	train's my_metric_4: 0.06982	valid's l2: 12.1153	valid's my_metric_4: 0.0776384
[112]	train's l2: 9.92175	train's my_metric_4: 0.0696404	valid's l2: 12.0713	valid's my_metric_4: 0.0774652
[113]	train's l2: 9.88802	train's my_metric_4: 0.0695023	valid's l2: 12.05	valid's my_metric_4: 0.0773732
[114]	train's l2: 9.83866	train's my_metric_4: 0.0693186	valid's l2: 12.0164	valid's my_metric_4: 0.0772385
[115]	train's l2: 9.80799	train's my_metric_4: 0.0691896	valid's l2: 11.9873	valid's my_metric_4: 0.0771372
[116]	train's l2: 9.76985	train's my_metric_4: 0.0690546	valid's l2: 11.9662	valid's my_metric_4: 0.0770466
[117]	train's l2: 9.71868	train's my_metric_4: 0.0688409	valid's l2: 11.9147	valid's my_metric_4: 0.0768675
[118]	train's l2: 9.68245	train's 

mae: [0.0708754597348987, 0.07042289000082064, 0.0712676534681158, 0.07273163751423427, 0.07062917287780449, 0.06821668741665932, 0.07273340242330403, 0.07120496030932907, 0.07295891977102861, 0.07121500455880671] 0.07122557880750016


In [69]:
train_features = X_train_4.columns
model_4.feature_importances_
pd.DataFrame({'features':train_features, 'imp':model_4.feature_importances_}).sort_values('imp',ascending=False)

,features,imp
27,辐照度日最大,593
6,时间戳,575
2,风向,472
45,压强日平均,453
11,湿度日最小,448
28,风速日最大,429
47,辐照度月平均,372
9,温度日最小,354
31,湿度日最大,342
8,风速日最小,334


In [70]:
prediction = pd.concat([prediction_1,prediction_2],axis=0,sort=False)
prediction = pd.concat([prediction,prediction_3],axis=0,sort=False)
prediction = pd.concat([prediction,prediction_4],axis=0,sort=False)
prediction

,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,prediction
0,1,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
1,2,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
2,3,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
3,4,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
4,5,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
5,6,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
6,7,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
7,8,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
8,9,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319
9,10,-0.004724,-0.005055,-0.005638,-0.005765,-0.005087,-0.005642,-0.004508,-0.006056,-0.005146,-0.005569,-0.005319


In [71]:
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')

prediction.rename(columns={'prediction':'predicition'},inplace=True)
prediction[['id', 'predicition']].to_csv("./result/baseline_%s.csv" % now, index=False)

import csv
import codecs

baseloss.append(np.mean(baseloss))
baseloss.insert(0,now)
file_csv = codecs.open('./result/baseline_valid_l1_loss.csv','a+','utf-8',buffering=0)#追加，缓存为0
# writer = csv.writer(file_csv, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
writer = csv.writer(file_csv)
writer.writerow(baseloss)